In [1]:
import torch
from torchtext import vocab
from dataset import SentimentAnalysisDataset
import pickle
from model import SentimentLSTM
from torch.nn import CrossEntropyLoss

loss = CrossEntropyLoss()

/Users/snirlugassy/miniconda3/envs/deep/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('vocab.text', 'rb') as pkl:
    vocab = pickle.load(pkl)

dataset = SentimentAnalysisDataset('data/trainEmotions.csv', vocab)
tokens, emotion = dataset[0]

model = SentimentLSTM(len(vocab))

In [4]:
x = model.embedding(tokens)
x.shape

torch.Size([11, 300])

In [5]:
x,_ = model.lstm(model.embedding(tokens))
x.shape

torch.Size([11, 400])

In [6]:
output = model.fc(x)
output.shape, output

(torch.Size([11, 3]),
 tensor([[-0.1283,  0.0213,  0.0023],
         [-0.1297,  0.0250,  0.0052],
         [-0.1271,  0.0224,  0.0036],
         [-0.1268,  0.0215,  0.0012],
         [-0.1231,  0.0217,  0.0051],
         [-0.1247,  0.0250,  0.0030],
         [-0.1209,  0.0241,  0.0016],
         [-0.1177,  0.0232,  0.0018],
         [-0.1152,  0.0215,  0.0032],
         [-0.1181,  0.0205,  0.0060],
         [-0.1237,  0.0193,  0.0065]], grad_fn=<AddmmBackward0>))

In [14]:
output[-1].shape, emotion.shape

(torch.Size([3]), torch.Size([3]))

In [30]:
loss(output[-1].view(1,-1), emotion.view(1,-1))

tensor(1.1917, grad_fn=<DivBackward1>)

In [ ]:
torch.Tensor([emotion])